In [1]:
%load_ext autoreload
%autoreload 2

# if you need to install Pybullet:
#   pip3 install Pybullet   (python3)
#   pip install PyBullet    (python2.x)

# let's start with imports
import numpy as np
from numpy.random import seed
from numpy.random import rand
import pybullet as p

from engine import *
from matplotlib import pyplot as plt

# seed random number generator.
seed(1)

%matplotlib inline

In [2]:
# create a pybullet engine from the engine module
engine = PyBulletEnvironment(gui=False)


In [3]:
#load shovel
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
# robot_urdf = './urdf/shovel/shovelFlat.urdf'
robot_urdf = './urdf/shovel/rover_with_shovel.urdf'

# load the robot
engine.open_environment(robot_urdf, startPos, startOrientation)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 24.0.9-0ubuntu0.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 24.0.9-0ubuntu0.2
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-P)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu


In [4]:
# load pebbles

# get shovel_link position
shovelLinkState = p.getLinkState(engine.ID[1], 2)

# admissible init positions
# minpos = np.asarray(shovelLinkState[0]) 
minpos = np.array([0.2,-0.4,0])
maxpos = np.array([0.7, 0.4, 0])
# maxpos = np.asarray(shovelLinkState[0]) + 4*1e-1

# number of pebbles
pebbleNum = 500

# scatter pebbles
for i in range (pebbleNum):
    
    # generate init pos
    startPos = minpos + (rand(3) * (maxpos - minpos))
    startPos[-1] = 0.2
    
    # set orientation
    startOrientation = p.getQuaternionFromEuler([0,0,0])
    
    # load pebble 
    engine.load_urdf('./urdf/pebbles/pebbles.urdf', startPos, startOrientation)    
    
# simulate
engine.simulate(0.5)

In [ ]:
import time
# Get joint indices for wheels
left_wheel_joint = 0  # Index for left wheel joint
right_wheel_joint = 1  # Index for right wheel joint

robot = engine.ID[1]
# Simulate
try:
    for _ in range(1000):  # Run for some steps
        # Set wheel velocities
        p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity=0, force=0)
        p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity=.0, force=0)
        p.setJointMotorControl2(robot, left_wheel_joint, p.VELOCITY_CONTROL, targetVelocity=-0.50, force=0.1)
        p.setJointMotorControl2(robot, right_wheel_joint, p.VELOCITY_CONTROL, targetVelocity=-4.0, force=0.1)

        p.stepSimulation()  # Advance simulation
        time.sleep(1. / 240.)  # Sleep to match real-time

# get the current end effector position
endEffectorPos = p.getLinkState(engine.ID[1], 2)[0]
pathLength = 0.5
pathCurvature = 5
Nwaypoints = 15

# Disconnect
# p.disconnect()

Initial position: (0.7, 0.0, 0.025)
Waypoints:
[0.7, 0.0, 0.025]
[0.7263095911936983, 0.0004945994409903953, 0.025]
[0.752582003218548, 0.001977698825084362, 0.025]
[0.7787801094451171, 0.004447202323349608, 0.025]
[0.8048668882485644, 0.007899620178501099, 0.025]
[0.8308054753254228, 0.012330073636421024, 0.025]
[0.8565592157880684, 0.017732301840527542, 0.025]
[0.8820917159632551, 0.024098670679249756, 0.025]
[0.9073668948215148, 0.031420183574105476, 0.025]
[0.9323490349647499, 0.03968649419313752, 0.025]
[0.9570028330999614, 0.04888592107174239, 0.025]
[0.9812934499277894, 0.05900546412022967, 0.025]
[1.0051865593753646, 0.07003082299478569, 0.025]
[1.0286483971038973, 0.08194641730587882, 0.025]
[1.0516458082224587, 0.09473540863555074, 0.025]
[1.0741462941405246, 0.10837972433248022, 0.025]
[1.0961180584930748, 0.122860083051192, 0.025]
[1.117530052073348, 0.1381560219993211, 0.025]
[1.1383520167097578, 0.1542459258544301, 0.025]
[1.158554528024963, 0.17110705730951237, 0.025]


In [7]:
img = engine.get_top_view()
im = np.array(img[2])
im = im.reshape((img[0],img[1], 4))
plt.figure(figsize=(10,10))
plt.imshow(im[:,:, :3])